<a href="https://colab.research.google.com/github/Matrix7043/Machine_learning101/blob/main/Softmax_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_iris

In [35]:
np.random.seed(69)

In [34]:
iris_data = load_iris()
iris_data.keys()
x_train = iris_data['data']
one = np.ones((150, 1))
x_train = np.hstack((x_train, one))
y_t = iris_data['target'].reshape(-1, 1)
f = (y_t == 0).reshape(-1)
g = (y_t == 1).reshape(-1)
h = (y_t == 2).reshape(-1)
y_train = np.stack((f, g, h), axis=1)

In [36]:
def denom(x):
    return 1 / np.sum(x, axis=1, keepdims=True)

def normalize(x):
    return x * denom(x)

def softmax(x):
    return normalize(np.exp(x))

In [93]:
def log_loss(y_train, prediction):
    loss = -1*y_train*np.log(prediction + 1e-6)
    loss = np.sum(loss, axis=1, keepdims=True)
    return loss

In [94]:
class SoftmaxRegression:

    def __init__(self, x, y, split, activation, loss, batch_size = None):
        self.x = x
        self.y = y
        ones = np.ones((self.x.shape[0],1))
        self.x = np.hstack((self.x,ones))

        split_ = int(len(x)*split)

        self.x = x[:split_]
        self.y = y[:split_]

        self.x_t = x[split_:]
        self.y_t = y[split_:]

        self.y_pred = np.zeros((self.y_t.shape[0], 1))

        self.w = np.random.rand(x.shape[1],1)
        self.batch_size = batch_size if batch_size else len(self.x)
        self.activation = activation
        self.loss = loss

    def forward(self, x_batch, y_batch):
        n = np.dot(x_batch, self.w)
        p = self.activation(n)

        c = self.loss(p, y_batch)
        return x_batch, y_batch, p

    def backward(self, x_batch, y_batch, p):
        dJ_dW = np.dot(x_batch.T,  p - y_batch)
        return dJ_dW/x_batch.shape[0]

    def fit(self, epoch, learning_rate):
        for i in range(len(self.x)//self.batch_size):
            x_batch = self.x[self.batch_size*i: self.batch_size*(i+1)]
            y_batch = self.y[self.batch_size*i: self.batch_size*(i+1)]
            for _ in range(epoch):
                x_batch, y_batch, p = self.forward(x_batch, y_batch)
                dJ_dw = self.backward(x_batch, y_batch, p)
                self.w = self.w - learning_rate * dJ_dw

    def predict(self):
        xt = self.x_t
        yt = self.y_t
        self.y_pred = self.forward(xt, yt)[2]
        return self.y_pred

    def accuracy(self):

        yp = np.zeros((self.y_pred.shape[0]))
        yt = np.zeros((self.y_pred.shape[0]))

        for i,j in enumerate(self.y_pred):
            yp[i] = np.argmax(j)
            yt[i] = np.argmax(self.y_t[i])

        yp.astype(int)

        acc = np.mean(yp == yt)

        return yp, acc


In [95]:
soft = SoftmaxRegression(x_train, y_train, 0.9, softmax, log_loss)

In [96]:
soft.fit(epoch=10000, learning_rate=0.001)

In [97]:
soft.predict()

array([[0.00260716, 0.42125956, 0.57613328],
       [0.004819  , 0.3206845 , 0.67449651],
       [0.00715346, 0.42733727, 0.56550927],
       [0.01871081, 0.49159461, 0.48969458],
       [0.00862763, 0.47513686, 0.51623552],
       [0.00408189, 0.35653705, 0.63938106],
       [0.01266103, 0.49985413, 0.48748484],
       [0.00597943, 0.36606417, 0.62795639],
       [0.00301596, 0.33884934, 0.6581347 ],
       [0.00399613, 0.33722667, 0.6587772 ],
       [0.00838961, 0.44170492, 0.54990547],
       [0.00729378, 0.44673529, 0.54597093],
       [0.00976534, 0.46039192, 0.52984273],
       [0.00737582, 0.35754152, 0.63508266],
       [0.01023562, 0.41947217, 0.5702922 ]])

In [98]:
soft.accuracy()[1]

0.8666666666666667